<a href="https://colab.research.google.com/github/mahaairshad/MyFirstRepos/blob/Feature-2/FBCSP_unsimplified_v5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install PyDrive

In [0]:
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
download = drive.CreateFile({'id': '1-GmBB-nwfaKqDaymIjJfS5HZVxS97be8'})
download.GetContentFile('DOWNLOAD.rar')

In [0]:
!pip install patool
import patoolib
patoolib.extract_archive("DOWNLOAD.rar")

     |████████████████████████████████| 81kB 2.6MB/s 
patool: Extracting DOWNLOAD.rar ...
patool: running /usr/bin/unrar x -- /content/DOWNLOAD.rar
patool:     with cwd='./Unpack_gpyuxcet'
patool: ... DOWNLOAD.rar extracted to `Data_sample'.


'Data_sample'

In [0]:
#Import Libraries
import numpy as np
import pandas as pd
from scipy.io import loadmat
from scipy.signal import butter, sosfilt, sosfreqz
from numpy import linalg as LA
from scipy import linalg as LAS
from sklearn.model_selection import GridSearchCV    # Import library for grid search
from sklearn import svm                             # Import SVM model library
from sklearn import metrics                         # Evaluating Accuracy
from sklearn.metrics import accuracy_score as acc
from sklearn.metrics import cohen_kappa_score       # Evaluating Kappa score
from sklearn.model_selection import cross_val_score # Cross Validation
from sklearn.metrics import make_scorer

### 1. Access dataset

In [0]:
# Loading single P's dataset
# s : (1,1)
# x : (3500,300,200)
# y : (1,200)
# c : ~(300,1)
# ci : ~(300,1)
# classInfo : (4,2)

In [0]:
annotsS1 = loadmat('Data_sample/ParsedMEGData_P1.mat')
annotsS2 = loadmat('Data_sample/ParsedMEGData_P1_S2.mat')

In [0]:
mdata1 = annotsS1['MEGdata']
mtype1 = mdata1.dtype
ndata1 = {n: mdata1[n][0,0] for n in mtype1.names}
#ndata1

In [0]:
s_sess1=ndata1['s']
x_sess1=ndata1['x']
y_sess1=ndata1['y']
c_sess1=ndata1['c']
ci_sess1=ndata1['ci']
class_info_Sess1=ndata1['classInfo']
#class_info_Sess1=
print(s_sess1.shape,'\n',x_sess1.shape,'\n', y_sess1.shape, '\n', c_sess1.shape, '\n',ci_sess1.shape)

(1, 1) 
 (3500, 300, 200) 
 (1, 200) 
 (300, 1) 
 (300, 1)


In [0]:
mdata2 = annotsS2['MEGdata']
mtype2 = mdata2.dtype
ndata2 = {n: mdata2[n][0,0] for n in mtype2.names}
#ndata2

In [0]:
s_sess2=ndata2['s']
x_sess2=ndata2['x']
y_sess2=ndata2['y']
c_sess2=ndata2['c']
ci_sess2=ndata2['ci']
class_info_Sess2=ndata2['classInfo']
#class_info_Sess2=
print(s_sess2.shape,'\n',x_sess2.shape,'\n', y_sess2.shape, '\n', c_sess2.shape, '\n',ci_sess2.shape)

(1, 1) 
 (3500, 300, 200) 
 (1, 200) 
 (300, 1) 
 (300, 1)


### 2. Considering the common channels from Session1 and Session2

In [0]:
cmn_chnls=np.intersect1d(ci_sess1,ci_sess2)

cmn_chnls=cmn_chnls.reshape(len(cmn_chnls),1) #reshaping so cmn_chnls, sess1_x and sess2_x have same shape

In [0]:
valid_chnl_ind1=np.array([],dtype=int)
valid_chnl_ind2=np.array([],dtype=int)

for i in range(0,len(ci_sess1)):    #Stores the index of common channels b/w sess1_x and cmn_chnls
    for j in range(0,len(cmn_chnls)):
        if ci_sess1[i,0]==cmn_chnls[j,0]:
            valid_chnl_ind1=np.append(valid_chnl_ind1,i)
            
for i in range(0,len(ci_sess2)):    #Stores the index of common channels b/w sess2_x and cmn_chnls
    for j in range(0,len(cmn_chnls)):
        if ci_sess2[i,0]==cmn_chnls[j,0]:
            valid_chnl_ind2=np.append(valid_chnl_ind2,i)

In [0]:
data1=x_sess1[:,valid_chnl_ind1,:] #(samples,channels,trials)
data2=x_sess2[:,valid_chnl_ind2,:] #(samples,channels,trials)

In [0]:
#Comibining session1 and session2 data (along 3rd axis(trials))
MEGdata_s= s_sess1
MEGdata_x= np.concatenate((data1,data2), axis=2)
MEGdata_y= np.concatenate((y_sess1,y_sess2), axis=1)

In [0]:
MEGdata_c= c_sess1[valid_chnl_ind1]
MEGdata_ci= cmn_chnls.copy()

In [0]:
print(MEGdata_s.shape,'\n',MEGdata_x.shape,'\n', MEGdata_y.shape, '\n', MEGdata_c.shape, '\n',MEGdata_ci.shape)

### 3. Implementation of Butterworth Filter

In [0]:
frqBand=pd.read_csv('frqBands.csv',sep=','
                    , header=None, index_col=None)
smpRange=pd.read_csv('smpRange.csv',sep=','
                    , header=None, index_col=None)
baseRange=pd.read_csv('baseRange.csv',sep=','
                    , header=None, index_col=None)
classProb=pd.read_csv('classProb.csv',sep=','
                    , header=None, index_col=None)

In [0]:
#Selecting Frequency band, Sampling range, Base range(noise) and Classes
FreqBand=np.array([frqBand.iloc[0]]) #band
FreqBand1=np.array([8, 12])
FreqBand2=np.array([13, 17])
FreqBand3=np.array([18, 22])
FreqBand4=np.array([24, 28])
SampRangeSec=np.array([smpRange.iloc[20]]) #actTs
BaseRangeSec=np.array([baseRange.iloc[1]]) #baseTs
Classes=np.array([classProb.iloc[0]]) #class1 class2

In [0]:
SampRange=s_sess1*SampRangeSec
SampRangeind=np.arange(SampRange[0,0],SampRange[0,1],1,dtype=int)#saves index for given range of time
BaseRange=s_sess1*BaseRangeSec
BaseRangeind=np.arange(BaseRange[0,0],BaseRange[0,1],1,dtype=int)#saves index for given range of time

In [0]:
#Separating data in classification of interest
cMEGdata_y_ind=np.array([],dtype=int)

for i in range(0,MEGdata_y.size): 
    if (MEGdata_y[0,i]==Classes[0,0])or(MEGdata_y[0,i]==Classes[0,1]):
        cMEGdata_y_ind=np.append(cMEGdata_y_ind,i)

In [0]:
cMEGdata_x=MEGdata_x[:,:,cMEGdata_y_ind] #(samples,channels,trials)
cMEGdata_y=MEGdata_y[0,cMEGdata_y_ind]

In [0]:
def butter_bandpass(lowcut, highcut, fs, order):
        nyq = 0.5 * fs
        low = lowcut / nyq
        high = highcut / nyq
        sos = butter(order, [low, high], analog=True, btype='bandpass', output='sos')
        return sos

def butter_bandpass_filter(data, lowcut, highcut, fs, order):
        sos = butter_bandpass(lowcut, highcut, fs, order=order)
        y = sosfilt(sos, data)
        return y

In [0]:
#(samples,channels,trials)--> (channels,trials,samples)
bMEGdata_x_1=np.zeros((cMEGdata_x[0,:,0].size,cMEGdata_x[0,0,:].size,SampRangeind.size))
bMEGdata_x_2=np.zeros((cMEGdata_x[0,:,0].size,cMEGdata_x[0,0,:].size,SampRangeind.size))
bMEGdata_x_3=np.zeros((cMEGdata_x[0,:,0].size,cMEGdata_x[0,0,:].size,SampRangeind.size))
bMEGdata_x_4=np.zeros((cMEGdata_x[0,:,0].size,cMEGdata_x[0,0,:].size,SampRangeind.size))

for trial in range(0,bMEGdata_x_1[0,:,0].size): #(channels,trials,samples) size of trial
    for channel in range(0,bMEGdata_x_1[:,0,0].size): #(channels,trials,samples) size of channel
        temp=cMEGdata_x[:,channel,trial] #(samples,channels,trials)
        temp=temp-np.mean(temp[BaseRangeind])
        f_temp1=butter_bandpass_filter(temp,FreqBand1[0],FreqBand1[1],MEGdata_s[0],4)
        f_temp1=f_temp1[SampRangeind]
        f_temp2=butter_bandpass_filter(temp,FreqBand2[0],FreqBand2[1],MEGdata_s[0],4)
        f_temp2=f_temp2[SampRangeind]
        f_temp3=butter_bandpass_filter(temp,FreqBand3[0],FreqBand3[1],MEGdata_s[0],4)
        f_temp3=f_temp3[SampRangeind]
        f_temp4=butter_bandpass_filter(temp,FreqBand4[0],FreqBand4[1],MEGdata_s[0],4)
        f_temp4=f_temp4[SampRangeind]
        bMEGdata_x_1[channel,trial,:]=f_temp1 #(channels,trials,samples)
        bMEGdata_x_2[channel,trial,:]=f_temp2
        bMEGdata_x_3[channel,trial,:]=f_temp3
        bMEGdata_x_4[channel,trial,:]=f_temp4

### 4.Separating train and validation data

Fband1

In [0]:
# Splitting train/test data in 50:50
size_Train_Test=int(bMEGdata_x_1[0,:,0].size/2)
bMEGdata_x_Train_1=bMEGdata_x_1[:,0:size_Train_Test,:]
bMEGdata_y_Train=cMEGdata_y[0:size_Train_Test]
bMEGdata_x_Test_1=bMEGdata_x_1[:,size_Train_Test:(size_Train_Test*2),:]
bMEGdata_y_Test=cMEGdata_y[size_Train_Test:(size_Train_Test*2)]

c1_bMEGdata_y_Train_ind=np.array([],dtype=int) #Getting index for class1 and class2 data
c2_bMEGdata_y_Train_ind=np.array([],dtype=int)

for i in range(0,bMEGdata_y_Train.size): 
    if (bMEGdata_y_Train[i]==Classes[0,0]):
        c1_bMEGdata_y_Train_ind=np.append(c1_bMEGdata_y_Train_ind,i)
    else:
        c2_bMEGdata_y_Train_ind=np.append(c2_bMEGdata_y_Train_ind,i)

# x and y values separated for class and 2
c1_bMEGdata_x_Train_1=bMEGdata_x_Train_1[:,c1_bMEGdata_y_Train_ind,:]
c1_bMEGdata_y_Train=bMEGdata_y_Train[c1_bMEGdata_y_Train_ind]
c2_bMEGdata_x_Train_1=bMEGdata_x_Train_1[:,c2_bMEGdata_y_Train_ind,:]
c2_bMEGdata_y_Train=bMEGdata_y_Train[c2_bMEGdata_y_Train_ind]

# Merging trial and sample dimensions into one: (channels,trials,samples)-->(channels,trials*samples)
c1_feat_Train_1=c1_bMEGdata_x_Train_1.reshape((c1_bMEGdata_x_Train_1[:,0,0].size,(c1_bMEGdata_x_Train_1[0,:,0].size*c1_bMEGdata_x_Train_1[0,0,:].size)))
c2_feat_Train_1=c2_bMEGdata_x_Train_1.reshape((c2_bMEGdata_x_Train_1[:,0,0].size,(c2_bMEGdata_x_Train_1[0,:,0].size*c2_bMEGdata_x_Train_1[0,0,:].size)))

c1_feat_Train_1.shape #(channels,trials*samples)

Fband2

In [0]:
# Splitting train/test data in 50:50
size_Train_Test=int(bMEGdata_x_2[0,:,0].size/2)
bMEGdata_x_Train_2=bMEGdata_x_2[:,0:size_Train_Test,:]
bMEGdata_y_Train=cMEGdata_y[0:size_Train_Test]
bMEGdata_x_Test_2=bMEGdata_x_2[:,size_Train_Test:(size_Train_Test*2),:]
bMEGdata_y_Test=cMEGdata_y[size_Train_Test:(size_Train_Test*2)]

c1_bMEGdata_y_Train_ind=np.array([],dtype=int) #Getting index for class1 and class2 data
c2_bMEGdata_y_Train_ind=np.array([],dtype=int)

for i in range(0,bMEGdata_y_Train.size): 
    if (bMEGdata_y_Train[i]==Classes[0,0]):
        c1_bMEGdata_y_Train_ind=np.append(c1_bMEGdata_y_Train_ind,i)
    else:
        c2_bMEGdata_y_Train_ind=np.append(c2_bMEGdata_y_Train_ind,i)

# x and y values separated for class and 2
c1_bMEGdata_x_Train_2=bMEGdata_x_Train_2[:,c1_bMEGdata_y_Train_ind,:]
c1_bMEGdata_y_Train=bMEGdata_y_Train[c1_bMEGdata_y_Train_ind]
c2_bMEGdata_x_Train_2=bMEGdata_x_Train_2[:,c2_bMEGdata_y_Train_ind,:]
c2_bMEGdata_y_Train=bMEGdata_y_Train[c2_bMEGdata_y_Train_ind]

# Merging trial and sample dimensions into one: (channels,trials,samples)-->(channels,trials*samples)
c1_feat_Train_2=c1_bMEGdata_x_Train_2.reshape((c1_bMEGdata_x_Train_2[:,0,0].size,(c1_bMEGdata_x_Train_2[0,:,0].size*c1_bMEGdata_x_Train_2[0,0,:].size)))
c2_feat_Train_2=c2_bMEGdata_x_Train_2.reshape((c2_bMEGdata_x_Train_2[:,0,0].size,(c2_bMEGdata_x_Train_2[0,:,0].size*c2_bMEGdata_x_Train_2[0,0,:].size)))

c1_feat_Train_2.shape #(channels,trials*samples)

Fband3

In [0]:
# Splitting train/test data in 50:50
size_Train_Test=int(bMEGdata_x_3[0,:,0].size/2)
bMEGdata_x_Train_3=bMEGdata_x_3[:,0:size_Train_Test,:]
bMEGdata_y_Train=cMEGdata_y[0:size_Train_Test]
bMEGdata_x_Test_3=bMEGdata_x_3[:,size_Train_Test:(size_Train_Test*2),:]
bMEGdata_y_Test=cMEGdata_y[size_Train_Test:(size_Train_Test*2)]

c1_bMEGdata_y_Train_ind=np.array([],dtype=int) #Getting index for class1 and class2 data
c2_bMEGdata_y_Train_ind=np.array([],dtype=int)

for i in range(0,bMEGdata_y_Train.size): 
    if (bMEGdata_y_Train[i]==Classes[0,0]):
        c1_bMEGdata_y_Train_ind=np.append(c1_bMEGdata_y_Train_ind,i)
    else:
        c2_bMEGdata_y_Train_ind=np.append(c2_bMEGdata_y_Train_ind,i)

# x and y values separated for class and 2
c1_bMEGdata_x_Train_3=bMEGdata_x_Train_3[:,c1_bMEGdata_y_Train_ind,:]
c1_bMEGdata_y_Train=bMEGdata_y_Train[c1_bMEGdata_y_Train_ind]
c2_bMEGdata_x_Train_3=bMEGdata_x_Train_3[:,c2_bMEGdata_y_Train_ind,:]
c2_bMEGdata_y_Train=bMEGdata_y_Train[c2_bMEGdata_y_Train_ind]

# Merging trial and sample dimensions into one: (channels,trials,samples)-->(channels,trials*samples)
c1_feat_Train_3=c1_bMEGdata_x_Train_3.reshape((c1_bMEGdata_x_Train_3[:,0,0].size,(c1_bMEGdata_x_Train_3[0,:,0].size*c1_bMEGdata_x_Train_3[0,0,:].size)))
c2_feat_Train_3=c2_bMEGdata_x_Train_3.reshape((c2_bMEGdata_x_Train_3[:,0,0].size,(c2_bMEGdata_x_Train_3[0,:,0].size*c2_bMEGdata_x_Train_3[0,0,:].size)))

c1_feat_Train_3.shape #(channels,trials*samples)

Fband4

In [0]:
# Splitting train/test data in 50:50
size_Train_Test=int(bMEGdata_x_4[0,:,0].size/2)
bMEGdata_x_Train_4=bMEGdata_x_4[:,0:size_Train_Test,:]
bMEGdata_y_Train=cMEGdata_y[0:size_Train_Test]
bMEGdata_x_Test_4=bMEGdata_x_4[:,size_Train_Test:(size_Train_Test*2),:]
bMEGdata_y_Test=cMEGdata_y[size_Train_Test:(size_Train_Test*2)]

c1_bMEGdata_y_Train_ind=np.array([],dtype=int) #Getting index for class1 and class2 data
c2_bMEGdata_y_Train_ind=np.array([],dtype=int)

for i in range(0,bMEGdata_y_Train.size): 
    if (bMEGdata_y_Train[i]==Classes[0,0]):
        c1_bMEGdata_y_Train_ind=np.append(c1_bMEGdata_y_Train_ind,i)
    else:
        c2_bMEGdata_y_Train_ind=np.append(c2_bMEGdata_y_Train_ind,i)

# x and y values separated for class and 2
c1_bMEGdata_x_Train_4=bMEGdata_x_Train_4[:,c1_bMEGdata_y_Train_ind,:]
c1_bMEGdata_y_Train=bMEGdata_y_Train[c1_bMEGdata_y_Train_ind]
c2_bMEGdata_x_Train_4=bMEGdata_x_Train_4[:,c2_bMEGdata_y_Train_ind,:]
c2_bMEGdata_y_Train=bMEGdata_y_Train[c2_bMEGdata_y_Train_ind]

# Merging trial and sample dimensions into one: (channels,trials,samples)-->(channels,trials*samples)
c1_feat_Train_4=c1_bMEGdata_x_Train_4.reshape((c1_bMEGdata_x_Train_4[:,0,0].size,(c1_bMEGdata_x_Train_4[0,:,0].size*c1_bMEGdata_x_Train_4[0,0,:].size)))
c2_feat_Train_4=c2_bMEGdata_x_Train_4.reshape((c2_bMEGdata_x_Train_4[:,0,0].size,(c2_bMEGdata_x_Train_4[0,:,0].size*c2_bMEGdata_x_Train_4[0,0,:].size)))

c1_feat_Train_4.shape #(channels,trials*samples)

### 5. Implementation of CSP

In [0]:
def CSP_projectionMat(class1, class2):
        #1 Find spatial covariance for both classes of data
        C1= np.matmul(class1,np.transpose(class1))/np.trace(np.matmul(class1,np.transpose(class1)))
        C2= np.matmul(class2,np.transpose(class2))/np.trace(np.matmul(class2,np.transpose(class2)))
        #2 Find composite spatial covariance
        Cc=C1+C2
        #3 Find eigen vectors and eigen values for Cc
        eigvalCc, eigvecCc= LA.eig(Cc)
        ## form eigval & eigvec in descending order and form diagonal matrix for eigval
        temp=np.sort(eigvalCc)  #sorts in ascending
        eigvalCc=temp[::-1]     #sorts in descending
        Cc_ind=np.argsort(temp) #finds arg for asc
        Cc_ind=Cc_ind[::-1]     #finds arg for des
        eigvalCc=np.diag(eigvalCc) #form diagonal matrix
        eigvecCc=eigvecCc[:,Cc_ind] #finds respective eigen vectors for the eig values
        #4 Find whitening transform
        P=np.matmul(LA.inv(eigvalCc),np.transpose(eigvecCc))
        P=np.sqrt(np.square(P)) #getting rid of the -ve values for simplicity here(MUST BE REMOVED)
        P=np.sqrt(P)
        #5 Find S1 and S2 for C1 and C2
        S1=np.matmul(P,C1) #S1=PC1P'
        S1=np.matmul(S1,np.transpose(P))
        S2=np.matmul(P,C2) #S2=PC2P'
        S2=np.matmul(S2,np.transpose(P))
        #6 Find generalized eigenvector and eigenvalues for S1 and S2
        eigvalB,B=LAS.eig(S1,S2, left=False, right=True)
        #7 Find W
        eigvalB=np.sort(eigvalB) #arranging in ascending to have the lowest eigenvalue on top and highest on bottom
        B_ind=np.argsort(eigvalB)
        B=B[:,B_ind]
        W=np.matmul(np.transpose(B),P)
        return W

In [0]:
W1=CSP_projectionMat(c1_feat_Train_1,c2_feat_Train_1)
W2=CSP_projectionMat(c1_feat_Train_2,c2_feat_Train_2)
W3=CSP_projectionMat(c1_feat_Train_3,c2_feat_Train_3)
W4=CSP_projectionMat(c1_feat_Train_4,c2_feat_Train_4)

In [0]:
# TRAIN DATA
#Finding 2 best features for each trial
Train_x_1=np.zeros((bMEGdata_x_Train_1[0,:,0].size,2)) #bMEGdata_x_Train=(channels,trials,samples)
Train_x_2=np.zeros((bMEGdata_x_Train_1[0,:,0].size,2))
Train_x_3=np.zeros((bMEGdata_x_Train_1[0,:,0].size,2))
Train_x_4=np.zeros((bMEGdata_x_Train_1[0,:,0].size,2))

for trial in range(0,bMEGdata_x_Train_1[0,:,0].size): #(channels,trials,samples) size of trial
    temp=bMEGdata_x_Train_1[:,trial,:] #(channels,trials,samples)-->(channels,samples)
    Z=np.matmul(np.transpose(W1),temp) #Z: CSP Features
    Feat=np.log(np.var(Z, axis=1)/np.sum(np.var(Z, axis=1))) #log of variance across each channel
    Train_x_1[trial,:]=[Feat[0], Feat[len(Feat)-1]] #2 best features representing each trial extracted

for trial in range(0,bMEGdata_x_Train_1[0,:,0].size): #(channels,trials,samples) size of trial
    temp=bMEGdata_x_Train_2[:,trial,:] #(channels,trials,samples)-->(channels,samples)
    Z=np.matmul(np.transpose(W2),temp) #Z: CSP Features
    Feat=np.log(np.var(Z, axis=1)/np.sum(np.var(Z, axis=1))) #log of variance across each channel
    Train_x_2[trial,:]=[Feat[0], Feat[len(Feat)-1]] #2 best features representing each trial extracted

for trial in range(0,bMEGdata_x_Train_1[0,:,0].size): #(channels,trials,samples) size of trial
    temp=bMEGdata_x_Train_3[:,trial,:] #(channels,trials,samples)-->(channels,samples)
    Z=np.matmul(np.transpose(W3),temp) #Z: CSP Features
    Feat=np.log(np.var(Z, axis=1)/np.sum(np.var(Z, axis=1))) #log of variance across each channel
    Train_x_3[trial,:]=[Feat[0], Feat[len(Feat)-1]] #2 best features representing each trial extracted

for trial in range(0,bMEGdata_x_Train_1[0,:,0].size): #(channels,trials,samples) size of trial
    temp=bMEGdata_x_Train_4[:,trial,:] #(channels,trials,samples)-->(channels,samples)
    Z=np.matmul(np.transpose(W4),temp) #Z: CSP Features
    Feat=np.log(np.var(Z, axis=1)/np.sum(np.var(Z, axis=1))) #log of variance across each channel
    Train_x_4[trial,:]=[Feat[0], Feat[len(Feat)-1]] #2 best features representing each trial extracted

Train_x=np.hstack([Train_x_1,Train_x_2])
Train_x.shape

In [0]:
# TEST DATA
#Finding 2 best features for each trial

Test_x_1=np.zeros((bMEGdata_x_Test_1[0,:,0].size,2)) #bMEGdata_x_Test=(channels,trials,samples)
Test_x_2=np.zeros((bMEGdata_x_Test_1[0,:,0].size,2))
Test_x_3=np.zeros((bMEGdata_x_Test_1[0,:,0].size,2))
Test_x_4=np.zeros((bMEGdata_x_Test_1[0,:,0].size,2))

for trial in range(0,bMEGdata_x_Test_1[0,:,0].size): #(channels,trials,samples) size of trial
    temp=bMEGdata_x_Test_1[:,trial,:] #(channels,trials,samples)-->(channels,samples)
    Z=np.matmul(np.transpose(W1),temp) #Z: CSP Features
    Feat=np.log(np.var(Z, axis=1)/np.sum(np.var(Z, axis=1))) #log of variance across each channel
    Test_x_1[trial,:]=[Feat[0], Feat[len(Feat)-1]] #2 best features representing each trial extracted

for trial in range(0,bMEGdata_x_Test_1[0,:,0].size): #(channels,trials,samples) size of trial
    temp=bMEGdata_x_Test_2[:,trial,:] #(channels,trials,samples)-->(channels,samples)
    Z=np.matmul(np.transpose(W2),temp) #Z: CSP Features
    Feat=np.log(np.var(Z, axis=1)/np.sum(np.var(Z, axis=1))) #log of variance across each channel
    Test_x_2[trial,:]=[Feat[0], Feat[len(Feat)-1]] #2 best features representing each trial extracted

for trial in range(0,bMEGdata_x_Test_1[0,:,0].size): #(channels,trials,samples) size of trial
    temp=bMEGdata_x_Test_3[:,trial,:] #(channels,trials,samples)-->(channels,samples)
    Z=np.matmul(np.transpose(W3),temp) #Z: CSP Features
    Feat=np.log(np.var(Z, axis=1)/np.sum(np.var(Z, axis=1))) #log of variance across each channel
    Test_x_3[trial,:]=[Feat[0], Feat[len(Feat)-1]] #2 best features representing each trial extracted

for trial in range(0,bMEGdata_x_Test_1[0,:,0].size): #(channels,trials,samples) size of trial
    temp=bMEGdata_x_Test_4[:,trial,:] #(channels,trials,samples)-->(channels,samples)
    Z=np.matmul(np.transpose(W4),temp) #Z: CSP Features
    Feat=np.log(np.var(Z, axis=1)/np.sum(np.var(Z, axis=1))) #log of variance across each channel
    Test_x_4[trial,:]=[Feat[0], Feat[len(Feat)-1]] #2 best features representing each trial extracted

Test_x=np.hstack([Test_x_1,Test_x_2])
Test_x.shape

In [0]:
Train_x.shape # X_train

In [0]:
Train_y=bMEGdata_y_Train #Y_train
Train_y.shape

In [0]:
Test_x.shape

In [0]:
Test_y=bMEGdata_y_Test
Test_y.shape

### 6. Classification

In [0]:
# SVM Classifier

In [0]:
def svc_param_selection(X, y, nfolds):
    parameters = {'kernel': ['rbf']},
    {'kernel': ['linear']}
    SVM = svm.SVC()
    grid_search = GridSearchCV(SVM, parameters, cv=nfolds) 
    grid_search.fit(X, y)
    grid_search.best_estimator_
    return grid_search.best_estimator_

SVM_params=svc_param_selection(Train_x,Train_y, 10)
print(SVM_params)

In [0]:
# Stating SVM model
SVM = svm.SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

# Training of SVM model using training set
SVM.fit(Train_x, Train_y)

# Predicting the trained model using the test set
svm_predictions = SVM.predict(Train_x) #test

### 7. Evaluation

In [0]:
#--------------Evaluating SVM--------------------------
print("Accuracy:",acc(Train_y, svm_predictions)*100)#test #Calculating the accuracy for SVM model

In [0]:
print("Kappa statistics: ",cohen_kappa_score(Train_y, svm_predictions))#test  # Calculating Kappa score for SVM

In [0]:
scores = cross_val_score(SVM, Train_x, Train_y, cv=10,scoring = make_scorer(acc))

print("ACC: %0.2f +/- %0.2f" % (scores.mean(), scores.std()))